In [ ]:
# This notebook is a testing playground
# Last Update: Always by Fei

In [1]:
cd /home/fwei16/tensorflow_privacy

/home/fwei16/privacy


In [ ]:
#=================================================
# Testing the @tf.function decorator and 
# tf.numpy_function
# Save gradients to .npy files
#=================================================
import tensorflow as tf
import numpy as np

dpsgd_type=0
var=0.25

filename = ('test_%d_v%1.2f.npy' %(dpsgd_type, var))
np.save(filename, np.empty(0))
print(np.size(np.load(filename)))

def my_numpy_func(grad):
    noise_norm=np.linalg.norm(grad)/np.sqrt(np.size(grad))
    filename = ('test_%d_v%1.2f.npy' %(dpsgd_type, var))
    buf=np.load(filename)
    buf=np.append(buf, noise_norm)
    np.save(filename, buf)
    return noise_norm.astype(np.float32)

@tf.function(input_signature=[tf.TensorSpec(None, tf.float32)])
def save_grad_norm(input):
    return tf.numpy_function(my_numpy_func, [input], tf.float32)

for _ in range(3):
    test_grad=tf.ones((3,4))
    save_grad_norm(test_grad)
    
print(np.load(filename))

In [ ]:
#=================================================
# TESTING FOR TENSORFLOW @tf.function, tf.numpy_function DECORATORS
#=================================================

import tensorflow as tf
import numpy as np

def my_numpy_func(grad):
    # x will be a numpy array with the contents of the input to the tf.function
#     size=np.shape(grad)
    stddev=0.5
    scale=stddev/np.sqrt(2)
    laplace=np.random.laplace(size=np.shape(grad), scale=0.4)
    return  np.ndarray.astype(laplace, np.float32)
        
@tf.function(input_signature=[tf.TensorSpec(None, tf.float32)])
def add_noise(input):
    y = tf.numpy_function(my_numpy_func, [input], tf.float32)
    return y 

grad=tf.zeros((2,3))
noise = add_noise(grad)
print(noise)

In [ ]:
#=================================================
# TESTING FOR READING .tfrecords FILES
#=================================================

import tensorflow as tf
import tempfile
tf.executing_eagerly()

filenames = ('privacy_data/sample_v%1.2f.tfrecords' %(var))

raw_dataset = tf.data.TFRecordDataset(filenames)

feature_description = {
    'laplace': tf.io.FixedLenSequenceFeature(shape=[], dtype=tf.float32, default_value=0.0, allow_missing = True),
    'cactus': tf.io.FixedLenSequenceFeature(shape=[], dtype=tf.float32, default_value=0.0, allow_missing = True),
}
def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    return parsed

parsed_dataset = raw_dataset.map(_parse_function)
iterator=iter(parsed_dataset)
iterator.get_next()

In [ ]:
#=================================================
# TESTING FOR WRITTING .tfrecords FILES
#=================================================
import numpy as np
import tensorflow as tf
from test_code.cactus_sampling import cactus_sample

var = 0.25
stddev = np.sqrt(var)
d = stddev/np.sqrt(2)
size = 30

filename=('privacy_data/sample_v%1.2f.tfrecords' %(var))
with tf.io.TFRecordWriter(filename) as writer:  
  for i in range(size):
    feature = {
          'laplace': tf.train.Feature(float_list=tf.train.FloatList(
              value=np.random.laplace(loc=0.0, size=1, scale=d))),
          'cactus': tf.train.Feature(float_list=tf.train.FloatList(
              value=cactus_sample(size=1, stddev=stddev))),
      }
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    writer.write(example.SerializeToString())
writer.close()


In [2]:
#=================================================
#   testing the TF model
#=================================================


import tensorflow as tf

input_shape=(250, 28, 28, 1)
x = tf.random.normal(input_shape)
print(x.shape)

x=tf.keras.layers.Conv2D(16, 8,
                       strides=2,
                       padding='same',
                       activation='relu',
                       input_shape=(28, 28, 1))(x)
print(x.shape)

x=tf.keras.layers.MaxPool2D(2, 1)(x)
print(x.shape)


x=tf.keras.layers.Conv2D(32, 4,
                       strides=2,
                       padding='valid',
                       activation='relu')(x)
print(x.shape)

x=tf.keras.layers.MaxPool2D(2, 1,padding='same')(x)
print(x.shape)

x=tf.keras.layers.Flatten()(x)
print(x.shape)

x=tf.keras.layers.Dense(32, activation='relu')(x)
print(x.shape)

x=tf.keras.layers.Dense(10)(x)
print(x.shape)


(250, 28, 28, 1)
(250, 14, 14, 16)
(250, 13, 13, 16)
(250, 5, 5, 32)
(250, 5, 5, 32)
(250, 800)
(250, 32)
(250, 10)


In [9]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from scipy import signal

stddev = 1
sen = 1

normal_dist0 = stats.norm(loc=0, scale=stddev)
normal_dist1 = stats.norm(loc=sen, scale=stddev)

delta = 1e-4
big_grid = np.arange(-8,8,delta)

pmfc = kernel.pdf(big_grid)*delta

pmf0 = normal_dist0.pdf(big_grid)*delta

pmf1 = normal_dist1.pdf(big_grid)*delta

conv_pmf0 = signal.convolve(pmfc,pmf0,'same')
conv_pmf1 = signal.convolve(pmfc,pmf1,'same')


In [32]:
#=================================================
#   testing the new cactus sampling function
#=================================================

import csv
import numpy as np


grad=np.zeros(shape=(2,3,4))

size=np.shape(grad)

def cactus_sample(stddev,size,sen):
  filename = ('privacy_data/cactus_samples_d%d_v%1.2f.npy' %(sen, stddev**2))
  samples=np.load(filename)

  result=np.zeros(size).reshape(-1)
  for a in range(len(result)):

    rand=np.random.uniform(0,len(samples),1)
    result[a]=samples[int(rand)]

  result=result.reshape(size)

  return result

result=cactus_sample(stddev=1,size=size,sen=1)
print(result)

[[[ 0.2792962   1.14857945  1.27760423 -4.22266871]
  [ 0.9232829  -1.27214551  2.44126153 -4.48867703]
  [-4.88676992  0.18202779  0.17846516  0.82104495]]

 [[-0.72686736  0.35453578  2.08125742 -1.79049484]
  [-1.0723485  -0.98825891 -1.18278631  2.25246997]
  [ 2.25834941 -2.69134059  1.04788004  0.89179998]]]
